In [ ]:
import os
import json
import requests
from bs4 import BeautifulSoup


In [ ]:
from tqdm import tqdm

In [ ]:
# set up a session
s = requests.session()

In [ ]:
def getData(url):
    response = requests.get(url)
    if(response.status_code==200):
        data = response.content
        jsonData = json.loads(data)
    else:
        print("Error occured", response.status_code())
    return jsonData

In [ ]:
#url1 = "https://do-ams3-17.hw.webhare.net/services/wobcovid19-prod-1/search/?first=1&count=2&orderby=publicationdate"
#url2 = "https://lin-euc-09.hw.webhare.net/services/wobcovid19-prod-2/search/"
url1 = "https://do-ams3-17.hw.webhare.net/services/wobcovid19-prod-1/search/?first=1&count=202&orderby=publicationdate"
data = getData(url1)

In [ ]:
# Serializing json  
#json_object = json.dumps(data, indent = 4) 
#print(json_object)

In [ ]:
import pandas as pd
# extract the list
df = pd.DataFrame() 
for i in tqdm(data["results"]):
    id = i["publication"]["id"][-32:].lower()
    pub_date = i["publication"]["publicationdate"][:10]
    title = i["publication"]["title"]
    df = df.append({'id':id, 'publication_date':pub_date, 'title':title}, ignore_index=True)
    

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
base_url = 'https://wobcovid19.rijksoverheid.nl/publicaties/'

In [ ]:
urls = base_url + df.id + '/'
urls

In [ ]:
def handle_filename_too_long(name):
    # os max char for filename on linux
    if name.startswith('besluit-op-wob-verzoek-'):
        name = name[23:len(name)]
    return name


In [ ]:
import random, string

def randomword(length):
   letters = string.ascii_lowercase
   return ''.join(random.choice(letters) for i in range(length))

In [ ]:
name= 'besluit-op-wob-verzoek-dd-22-juli-2021-inzake-kosten-en-batenanalyse-tabel-in-de-hypothetische-situatie-waarbij-deze-coronapandemie-ongecontroleerd-door-het-land-zou-razen-nav-documentaire-over-het-verloren-aantal-kwalitatieve-levensjaren-zogenoemde-qalys.pdf'
len(name)

if name.startswith('besluit-op-wob-verzoek-'):
    name[23:len(name)]

name[23:len(name)]

In [ ]:
for u in tqdm(urls):
    # pagina met wob docs
    r = requests.get(url = u).text
    # parse html
    soup = BeautifulSoup(r, 'html.parser')
    # looking for hrefs 'publication__download'
    hrefs = [a.get('href') for a in soup.find_all('a', attrs={"class":"publication__download"})]
    
    for h in hrefs:
        # pdfs in pagina
        r = requests.get(u + h, stream=True)
        
        # besluit.pdf komt meerdere keren voor we willen niet overschrtrijven
        if (h == 'besluit.pdf'):
            h = "besluit-" + randomword(5) + ".pdf"
        
        # handle long filename
        if len(h) > 255 :
            h = handle_filename_too_long(h)
            
        path = './data/'+ h
        chunk_size = 2000

        try :
            with open(path, 'wb') as fd:
                for chunk in r.iter_content(chunk_size):
                    fd.write(chunk)
        except OSError as exc:
            if exc.errno == 36:
                print('filename to long : ' + h)
            else:
                raise  # re-raise previously caught exception
